# Lab: Advanced Agent Features - Multi-Model Usage and Guardrails

This lab builds upon our previous multi-agent system and introduces three advanced features of the OpenAI Agents SDK:

1.  **Multi-Model Integration**: We will configure our agents to use models from different providers (like Google, DeepSeek, and Groq) instead of just OpenAI.
2.  **Structured Outputs**: We'll use Pydantic models to force an agent's output into a specific, predictable JSON structure.
3.  **Input Guardrails**: We will implement a safety mechanism to inspect and potentially block a user's prompt before it's processed by the main agent, ensuring responsible AI behavior.

In [ ]:
# === Imports ===
# Note the new imports from the agents SDK for advanced features.
import os
import asyncio
from typing import Dict
from dotenv import load_dotenv
from pydantic import BaseModel

# Core SDK components
from agents import Agent, Runner, trace, function_tool

# Advanced SDK components for multi-model support and guardrails
from agents import OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput

# The standard OpenAI client is now asynchronous for better performance with multiple models.
from openai import AsyncOpenAI

# SendGrid for our email tool
import sendgrid
from sendgrid.helpers.mail import Mail, Email, To, Content

In [ ]:
load_dotenv(override=True)

### Part 1: Integrating Multiple Model Providers

The OpenAI Agents SDK is designed to be model-agnostic. We can use any model that has an OpenAI-compatible API endpoint. Here, we'll set up clients for DeepSeek, Google's Gemini, and Llama 3 via Groq.

In [ ]:
# === API Key Verification ===
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

print(f"Google API Key set: {bool(google_api_key)}")
print(f"DeepSeek API Key set: {bool(deepseek_api_key)}")
print(f"Groq API Key set: {bool(groq_api_key)}")

In [ ]:
# === Configure Clients and Models for Different Providers ===

# We use AsyncOpenAI to create clients pointed at the specific API endpoints for each provider.
deepseek_client = AsyncOpenAI(base_url="https://api.deepseek.com/v1", api_key=deepseek_api_key)
gemini_client = AsyncOpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/openai/", api_key=google_api_key)
groq_client = AsyncOpenAI(base_url="https://api.groq.com/openai/v1", api_key=groq_api_key)

# We then wrap these clients in an `OpenAIChatCompletionsModel` object.
# This allows the Agents SDK to treat them just like a standard OpenAI model.
deepseek_model = OpenAIChatCompletionsModel(model="deepseek-chat", openai_client=deepseek_client)
gemini_model = OpenAIChatCompletionsModel(model="gemini-1.5-flash-latest", openai_client=gemini_client)
llama3_model = OpenAIChatCompletionsModel(model="llama3-70b-8192", openai_client=groq_client)

In [ ]:
# === Re-define our Sales Agents with the new models ===
company_description = "a company that provides a SaaS tool called 'ComplAI' for ensuring SOC2 compliance and preparing for audits, powered by AI."
instructions1 = f"You are a highly professional sales agent for {company_description} You write formal, serious, and benefit-driven cold emails."
instructions2 = f"You are a witty and engaging sales agent for {company_description} You write humorous, personable cold emails that are likely to get a response."
instructions3 = f"You are a busy, no-nonsense sales agent for {company_description} You write concise, to-the-point cold emails that respect the reader's time."

# Each agent is now powered by a different model provider.
sales_agent1 = Agent(name="DeepSeek_Sales_Agent", instructions=instructions1, model=deepseek_model)
sales_agent2 = Agent(name="Gemini_Sales_Agent", instructions=instructions2, model=gemini_model)
sales_agent3 = Agent(name="Llama3_Sales_Agent", instructions=instructions3, model=llama3_model)

### Part 2: Assembling the Agentic Workflow with New Models

The rest of our agentic workflow (the tools, the emailer agent, and the manager agent) remains largely the same. We just need to plug our new multi-provider agents into the system.

In [ ]:
# === Define Tools and the Emailer Agent ===

# This section is identical to the previous lab, defining the tools needed for the emailer agent.
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """Sends an email with a subject and HTML body."""
    FROM_EMAIL = "your-verified-sender@example.com"
    TO_EMAIL = "your-recipient@example.com"
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    mail = Mail(Email(FROM_EMAIL), To(TO_EMAIL), subject, Content("text/html", html_body))
    sg.client.mail.send.post(request_body=mail.get())
    return {"status": "success"}

subject_writer = Agent(name="Subject_Writer", instructions="You write compelling subjects for cold sales emails.", model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Writes a subject for an email based on its body.")

html_converter = Agent(name="HTML_Converter", instructions="You convert plain text to professional HTML.", model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter", tool_description="Converts a plain text email body to HTML.")

emailer_instructions = """You are an email formatting and sending specialist. You receive the body of an email. First, use the `subject_writer` tool to create a subject. Second, use the `html_converter` tool to format the body as HTML. Finally, use the `send_html_email` tool to send the email."""
emailer_agent = Agent(
    name="Emailer_Agent",
    instructions=emailer_instructions,
    tools=[subject_tool, html_tool, send_html_email],
    model="gpt-4o",
    handoff_description="Takes a plain text email body, formats it, and sends it."
)

In [ ]:
# === Assemble the Manager's Tools and Handoffs ===

# The manager's tools are our new multi-provider agents.
tools = [
    sales_agent1.as_tool(tool_name="deepseek_writer", tool_description="Writes a professional email."),
    sales_agent2.as_tool(tool_name="gemini_writer", tool_description="Writes an engaging email."),
    sales_agent3.as_tool(tool_name="llama_writer", tool_description="Writes a concise email.")
]

handoffs = [emailer_agent]

### Part 3: Implementing an Input Guardrail

A guardrail is a safety check. An **input guardrail** inspects the user's prompt *before* the main agent runs. We will create a guardrail to detect if a personal name is included in the prompt, which might violate privacy or usage policies.

In [ ]:
# === Define a Structured Output for the Guardrail Agent ===
# We use a Pydantic model to define the exact JSON structure we want back.
class NameCheckOutput(BaseModel):
    is_name_in_message: bool
    name_found: str

In [ ]:
# === Define the Guardrail Agent ===
# This agent's only job is to check for a name and return a structured response.
guardrail_agent = Agent( 
    name="Name_Check_Agent",
    instructions="Check if the user's prompt contains a person's first or last name. For example, 'Alice' or 'Smith'. If a name is found, set is_name_in_message to true and return the name.",
    output_type=NameCheckOutput, # This tells the agent to use the Pydantic model for its output.
    model="gpt-4o-mini"
)

In [ ]:
# === Define the Guardrail Function ===
# The `@input_guardrail` decorator registers this function as a safety check.
# It runs before the main agent logic.

@input_guardrail
async def guardrail_against_name(ctx, agent, message):
    print("--- Running Input Guardrail: Checking for names... ---")
    result = await Runner.run(guardrail_agent, message)
    
    # The `tripwire_triggered` flag tells the Runner to stop execution if it's True.
    is_name_present = result.final_output.is_name_in_message
    if is_name_present:
        print(f"!!! GUARDRAIL TRIGGERED: Name '{result.final_output.name_found}' found in prompt. Halting execution. !!!")
    else:
        print("--- Guardrail Passed: No name found. ---")
        
    return GuardrailFunctionOutput(
        output_info={"name_check_details": result.final_output},
        tripwire_triggered=is_name_present
    )

### Part 4: Running the Final, Guarded Workflow

Now we create our final Sales Manager agent, this time including the `input_guardrails` parameter. We will then test it with two different prompts: one that should be blocked by the guardrail, and one that should pass.

In [ ]:
# === Define the Final, Guarded Manager Agent ===
sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using your writer tools, and then hand it off to the Emailer Agent to be sent.
"""

careful_sales_manager = Agent(
    name="Careful_Sales_Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o",
    input_guardrails=[guardrail_against_name] # Add the guardrail here.
)

In [ ]:
# === Test 1: This prompt should be BLOCKED by the guardrail ===
message_with_name = "Send out a cold sales email addressed to Dear CEO from Alice"

print("--- RUNNING TEST 1 (SHOULD BE BLOCKED) ---")
with trace("Guarded_SDR_Blocked"):
    result = await Runner.run(careful_sales_manager, message_with_name)
    # Because the guardrail is triggered, the agent's main logic will not execute.
    # The `result.final_output` will likely be None or an empty response.

In [ ]:
# === Test 2: This prompt should PASS the guardrail ===
message_without_name = "Send out a cold sales email addressed to Dear CEO from the Head of Business Development"

print("\n\n--- RUNNING TEST 2 (SHOULD PASS) ---")
with trace("Guarded_SDR_Passed"):
    result = await Runner.run(careful_sales_manager, message_without_name)
    # This time, the full workflow should execute, and an email should be sent.

### Final Check

Remember to check two places:

1.  **The Traces**: [https://platform.openai.com/traces](https://platform.openai.com/traces). Look for the `Guarded_SDR_Blocked` and `Guarded_SDR_Passed` traces. In the first, you'll see the guardrail trigger and halt. In the second, you'll see the full, multi-agent execution.
2.  **Your Email Inbox**: To see the final email sent from the successful run.